# 字符串匹配——KMP算法

[字符串匹配的KMP算法-阮一峰](http://www.ruanyifeng.com/blog/2013/05/Knuth%E2%80%93Morris%E2%80%93Pratt_algorithm.html)

[从头到尾彻底理解KMP（2014年8月22日版）-July](https://blog.csdn.net/v_july_v/article/details/7041827)

有一个字符串"BBC ABCDAB ABCDABCDABDE"，我想知道，里面是否包含另一个字符串"ABCDABD"？

## 1.暴力匹配方法Brute Force(BF)算法：

 如果用暴力匹配的思路，并假设现在文本串S匹配到 i 位置，模式串P匹配到 j 位置，则有：

如果当前字符匹配成功（即S[i] == P[j]），则i++，j++，继续匹配下一个字符；

如果失配（即S[i]! = P[j]），令i = i - (j - 1)，j = 0。相当于每次匹配失败时，i 回溯并向前移动一位，j 被置为0。

代码：

In [1]:
def BFAlgo(s,p):
    slen = len(s)
    plen = len(p)
    i = j = 0
    while i < slen and j < plen:
        if s[i] == p[j]: #匹配到则继续往下匹配
            i += 1
            j += 1 #i=slen或者j=plen时退出while循环
        else: # 不匹配则回溯
            i = i - j + 1
            j = 0
    if j == plen:
        return i - j
    else:
        return -1

s = "abbaabbaaba"
p = "abbaaba"
BFAlgo(s,p)

4

## 2.KMP算法

### [字符串匹配的KMP算法-阮一峰](http://www.ruanyifeng.com/blog/2013/05/Knuth%E2%80%93Morris%E2%80%93Pratt_algorithm.html)

#### 1.部分匹配表

针对搜索词，算出一张《部分匹配表》（Partial Match Table）,LPS（longest prefix suffix） 最长前缀后缀公共元素长度

![](http://www.ruanyifeng.com/blogimg/asset/201305/bg2013050109.png)


"部分匹配值"就是"前缀"和"后缀"的最长的共有元素的长度。以"ABCDABD"为例，


－　"A"的前缀和后缀都为空集，共有元素的长度为0；

－　"AB"的前缀为[A]，后缀为[B]，共有元素的长度为0；

－　"ABC"的前缀为[A, AB]，后缀为[BC, C]，共有元素的长度0；

－　"ABCD"的前缀为[A, AB, ABC]，后缀为[BCD, CD, D]，共有元素的长度为0；

－　"ABCDA"的前缀为[A, AB, ABC, ABCD]，后缀为[BCDA, CDA, DA, A]，共有元素为"A"，长度为1；

－　"ABCDAB"的前缀为[A, AB, ABC, ABCD, ABCDA]，后缀为[BCDAB, CDAB, DAB, AB, B]，共有元素为"AB"，长度为2；

－　"ABCDABD"的前缀为[A, AB, ABC, ABCD, ABCDA, ABCDAB]，后缀为[BCDABD, CDABD, DABD, ABD, BD, D]，共有元素的长度为0。

#### 2.匹配，移动

先找到和待匹配字符串第一位相同的字符，一直匹配到第一个不匹配的字符。

![](http://www.ruanyifeng.com/blogimg/asset/201305/bg2013050107.png)

已知空格与D不匹配时，前面六个字符"ABCDAB"是匹配的。查表可知，最后一个匹配字符B对应的"部分匹配值"为2，因此按照下面的公式算出向后移动的位数：

```
移动位数 = 已匹配的字符数 - 对应的部分匹配值
```

因为 6 - 2 等于4，所以将搜索词向后移动4位。

![](http://www.ruanyifeng.com/blogimg/asset/201305/bg2013050110.png)

可以看出，p串最前面的两个AB移动到了对应于s串的AB的位置。

### [从头到尾彻底理解KMP（2014年8月22日版）-July](https://blog.csdn.net/v_july_v/article/details/7041827)

假设现在文本串S匹配到 i 位置，模式串P匹配到 j 位置

- 如果j = -1，或者当前字符匹配成功（即S[i] == P[j]），都令i++，j++，继续匹配下一个字符；
- 如果j != -1，且当前字符匹配失败（即S[i] != P[j]），则令 i 不变，j = next[j]。此举意味着失配时，模式串P相对于文本串S向右移动了j - next [j] 位。
    - 换言之，当匹配失败时，模式串向右移动的位数为：失配字符所在位置 - 失配字符对应的next 值（next 数组的求解会在下文的3.3.3节中详细阐述），即移动的实际位数为：j - next[j]，且此值大于等于1。
    
很快，你也会意识到next 数组各值的含义：代表当前字符之前的字符串中，有多大长度的相同前缀后缀。例如如果next [j] = k，代表j 之前的字符串中有最大长度为k 的相同前缀后缀。

此也意味着在某个字符失配时，该字符对应的next 值会告诉你下一步匹配中，模式串应该跳到哪个位置（跳到next [j] 的位置）。如果next [j] 等于0或-1，则跳到模式串的开头字符，若next [j] = k 且 k > 0，代表下次匹配跳到j 之前的某个字符，而不是跳到开头，且具体跳过了k 个字符。

转换成代码表示(nextArr求解见下文)：

In [2]:
def KMPAlgo(s,p):
    i=j=0
    slen = len(s)
    plen = len(p)
    while i < slen and j < plen:
        #如果j = -1，或者当前字符匹配成功（即S[i] == P[j]），都令i++，j++  
        if j==-1 or s[i] == s[j]:
            i += 1
            j += 1
        #如果j != -1，且当前字符匹配失败（即S[i] != P[j]），则令 i 不变，j = next[j]
        else:
            j = nextArr[j] #nextArr[j]即为j所对应的next值
    if j == plen:
        return i - j
    else:
        return -1

**步骤：**

#### 1.寻找前缀后缀最长公共元素长度

如abab为0，0，1，2

#### 2.求next数组

next 数组考虑的是除当前字符外的最长相同前缀后缀，所以通过第1步骤求得各个前缀后缀的公共元素的最大长度后，只要稍作变形即可：将第1步骤中求得的值整体右移一位，然后初值赋为-1，如下表格所示：

-1，0，0，1

如对于aba来说，第3个字符a之前的字符串ab中有长度为0的相同前缀后缀，所以第3个字符a对应的next值为0；

而对于abab来说，第4个字符b之前的字符串aba中有长度为1的相同前缀后缀a，所以第4个字符b对应的next值为1。

#### 3.根据next数组进行匹配

![](https://img-blog.csdn.net/20140812223633281)

 综上，KMP的next 数组相当于告诉我们：当模式串中的某个字符跟文本串中的某个字符匹配失配时，模式串下一步应该跳到哪个位置。如模式串中在j 处的字符跟文本串在i 处的字符匹配失配时，下一步用next [j] 处的字符继续跟文本串i 处的字符匹配，相当于模式串向右移动 j - next[j] 位。

In [3]:
# next数组计算
def getNext(p):
    plen = len(p)
    nextArr = [0]*plen
    nextArr[0] = -1
    k = -1
    j = 0
    while j < plen-1: # p[k]表示前缀，p[j]表示后缀
        if k == -1 or p[j] == p[k]:
            k += 1
            j += 1
            nextArr[j] = k
        else:
            k = nextArr[k]
    return nextArr

getNext("ababb")

[-1, 0, 0, 1, 2]

#### 4.Next 数组的优化

 行文至此，咱们全面了解了暴力匹配的思路、KMP算法的原理、流程、流程之间的内在逻辑联系，以及next 数组的简单求解（《最大长度表》整体右移一位，然后初值赋为-1）和代码求解，最后基于《next 数组》的匹配，看似洋洋洒洒，清晰透彻，但以上忽略了一个小问题。

 比如，如果用之前的next 数组方法求模式串“abab”的next 数组，可得其next 数组为-1 0 0 1（0 0 1 2整体右移一位，初值赋为-1），当它跟下图中的文本串去匹配的时候，发现b跟c失配，于是模式串右移j - next[j] = 3 - 1 =2位。

![](http://hi.csdn.net/attachment/201106/14/8394323_1308075859Zfue.jpg)

右移2位后，b又跟c失配。事实上，因为在上一步的匹配中，已经得知p[3] = b，与s[3] = c失配，而右移两位之后，让p[ next[3] ] = p[1] = b 再跟s[3]匹配时，必然失配。问题出在哪呢？

![](http://hi.csdn.net/attachment/201106/14/8394323_13080758591kyV.jpg)

问题出在不该出现p[j] = p[ next[j] ]。为什么呢？理由是：当p[j] != s[i] 时，下次匹配必然是p[ next [j]] 跟s[i]匹配，如果p[j] = p[ next[j] ]，必然导致后一步匹配失败（因为p[j]已经跟s[i]失配，然后你还用跟p[j]等同的值p[next[j]]去跟s[i]匹配，很显然，必然失配），所以不能允许p[j] = p[ next[j ]]。如果出现了p[j] = p[ next[j] ]咋办呢？如果出现了，则需要再次递归，即令next[j] = next[ next[j] ]。

所以，咱们得修改下求next 数组的代码。


In [4]:
# 优化过后的next 数组求法
def getNext(p):
    plen = len(p)
    nextArr = [0]*plen
    nextArr[0] = -1
    k = -1
    j = 0
    while j < plen-1: # p[k]表示前缀，p[j]表示后缀
        if k == -1 or p[j] == p[k]:
            k += 1
            j += 1
            #较之前next数组求法，改动在下面4行
            if p[j] != p[k]:
                nextArr[j] = k
            else:
                #因为不能出现p[j] = p[ next[j ]]，所以当出现时需要继续递归，k = next[k] = next[next[k]]
                nextArr[j] = nextArr[k]
        else:
            k = nextArr[k]
    return nextArr

getNext("ababcabab")

[-1, 0, -1, 0, 2, -1, 0, -1, 0]

对于优化后的next数组可以发现一点：如果模式串的后缀跟前缀相同，那么它们的next值也是相同的，例如模式串abcabc，它的前缀后缀都是abc，其优化后的next数组为：-1 0 0 -1 0 0，前缀后缀abc的next值都为-1 0 0。

## 3.KMP算法完整代码

In [5]:
# 优化过后的next 数组求法
def getNext(p):
    plen = len(p)
    nextArr = [0]*plen
    nextArr[0] = -1
    k = -1
    j = 0
    while j < plen-1: # p[k]表示前缀，p[j]表示后缀
        if k == -1 or p[j] == p[k]:
            k += 1
            j += 1
            #较之前next数组求法，改动在下面4行
            if p[j] != p[k]:
                nextArr[j] = k
            else:
                #因为不能出现p[j] = p[ next[j ]]，所以当出现时需要继续递归，k = next[k] = next[next[k]]
                nextArr[j] = nextArr[k]
        else:
            k = nextArr[k]
    return nextArr

def KMPAlgo(s,p):
    #KMP算法
    i = j = 0
    slen = len(s)
    plen = len(p)
    nextArr = getNext(p)
    while i < slen and j < plen:
        #如果j = -1，或者当前字符匹配成功（即S[i] == P[j]），都令i++，j++  
        if j==-1 or s[i] == p[j]:
            i += 1
            j += 1
        #如果j != -1，且当前字符匹配失败（即S[i] != P[j]），则令 i 不变，j = next[j]
        else:
            j = nextArr[j] #nextArr[j]即为j所对应的next值
    if j == plen:
        return i - j
    else:
        return -1

if __name__=="__main__":
    s = "BBC ABCDAB ABCDABCDABDE"
    p = "ABCDABD"
    print(KMPAlgo(s,p))

15
